In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import sys

sys.path.append("..")
from model import Net

In [4]:
def get_model_params(model):
    total_params = sum(p.numel() for p in model.parameters())
    print(f'Total Parameters: {total_params}')
    return total_params

model = Net()
get_model_params(model)


Total Parameters: 16034


16034

In [5]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
         Dropout2d-3           [-1, 16, 26, 26]               0
         ConvBlock-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
       BatchNorm2d-6           [-1, 16, 24, 24]              32
         Dropout2d-7           [-1, 16, 24, 24]               0
         ConvBlock-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 32, 22, 22]           4,640
      BatchNorm2d-10           [-1, 32, 22, 22]              64
        Dropout2d-11           [-1, 32, 22, 22]               0
        ConvBlock-12           [-1, 32, 22, 22]               0
        MaxPool2d-13           [-1, 32, 11, 11]               0
           Conv2d-14            [-1, 8,

In [6]:
torch.manual_seed(1)
batch_size = 128

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    transforms.RandomAffine(degrees=0, shear=10) #shear by 10 degrees
])

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transform),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform),
    batch_size=batch_size, 
    shuffle=True, 
    **kwargs
)


In [7]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        accuracy = (output.argmax(dim=1) == target).float().mean()
        loss.backward()

        total_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)

        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch}; loss={loss.item():.4f}; accuracy={accuracy.item():.3f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
epochs = 20

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch=1; loss=0.1493; accuracy=0.958: 100%|██████████| 469/469 [00:33<00:00, 13.91it/s]


Test set: Average loss: 0.0944, Accuracy: 9747/10000 (97.47%)



epoch=2; loss=0.0549; accuracy=0.979: 100%|██████████| 469/469 [00:35<00:00, 13.04it/s]


Test set: Average loss: 0.0549, Accuracy: 9836/10000 (98.36%)



epoch=3; loss=0.0497; accuracy=0.990: 100%|██████████| 469/469 [00:30<00:00, 15.16it/s]


...

Test set: Average loss: 0.0192, Accuracy: 9933/10000 (99.33%)



epoch=19; loss=0.0702; accuracy=0.969: 100%|██████████| 469/469 [00:32<00:00, 14.53it/s]


Test set: Average loss: 0.0231, Accuracy: 9922/10000 (99.22%)



epoch=20; loss=0.0215; accuracy=0.990: 100%|██████████| 469/469 [00:32<00:00, 14.58it/s]


Test set: Average loss: 0.0200, Accuracy: 9935/10000 (99.35%)

